In [1]:
# IMPORT LIBRARIES
##################

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')


# VISUAL SETTINGS
##################

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("viridis")



# LOADING CSV
##################

def load_latest_csv(data_dir='data', rename_files=False):
    """Load the most recent CSV file from the data directory"""
    files = [
        os.path.join(data_dir, f)
        for f in os.listdir(data_dir)
        if os.path.isfile(os.path.join(data_dir, f)) and f.endswith('.csv')
    ]

    sorted_files = sorted(files, key=os.path.getctime, reverse=True)

    for file_path in sorted_files:
        created_at = datetime.fromtimestamp(os.path.getctime(file_path))
        timestamp_str = created_at.strftime('%Y-%m-%d_%H-%M-%S')
        original_name = os.path.basename(file_path)
        new_name = f"{timestamp_str}_{original_name}"
        new_path = os.path.join(data_dir, new_name)

        if rename_files:
            if not original_name.startswith(timestamp_str):
                os.rename(file_path, new_path)
                print(f"Renamed: {original_name} → {new_name}")
                file_path = new_path
            else:
                print(f"Already renamed: {original_name}")
        else:
            print(f"[DEV] Would rename: {original_name} → {new_name}")

    return pd.read_csv(sorted_files[0])

# Load data
df_raw = load_latest_csv()




# DATA PREPROCESSING
##################

def preprocess_data(df):
    """Clean and preprocess the subscription data"""
    df = df.copy()

    # Date conversion
    date_cols = [col for col in df.columns if '(UTC)' in col]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

    df = df.sort_values(by='Created (UTC)')

    # Column selection and renaming
    columns_to_keep = [
        'Customer Name', 'Status', 'Cancellation Reason',
        'Created (UTC)', 'Start (UTC)', 'Start Date (UTC)', 
        'Current Period Start (UTC)', 'Current Period End (UTC)', 
        'Trial Start (UTC)', 'Trial End (UTC)',
        'Canceled At (UTC)', 'Ended At (UTC)', 
        'senderShopifyCustomerId (metadata)'
    ]
    
    df = df[columns_to_keep]

    df.rename(columns={
        'Customer ID': 'customer_id',
        'Customer Name': 'customer_name',
        'Status': 'status',
        'Created (UTC)': 'created_utc',
        'Start (UTC)': 'start_utc',
        'Start Date (UTC)': 'start_date_utc',
        'Current Period Start (UTC)': 'current_period_start_utc',
        'Current Period End (UTC)': 'current_period_end_utc',
        'Trial Start (UTC)': 'trial_start_utc',
        'Trial End (UTC)': 'trial_end_utc',
        'Canceled At (UTC)': 'canceled_at_utc',
        'Ended At (UTC)': 'ended_at_utc',
        'senderShopifyCustomerId (metadata)': 'is_gifted_member'
    }, inplace=True)

    # Convert is_gifted_member to boolean
    df['is_gifted_member'] = df['is_gifted_member'].notna()

    # Reference date for analysis
    reference_date = pd.Timestamp.now(tz='UTC')
    print(f"📅 Reference date (TODAY) for analysis: {reference_date.strftime('%d-%m-%Y')}")

    # Consolidate status
    df.loc[df['status'].isin(['past_due', 'incomplete_expired']), 'status'] = 'canceled'

    return df, reference_date

df, reference_date = preprocess_data(df_raw)



[DEV] Would rename: DishpatchSubscriptionData_NIklas_Sanitised - subscriptions (2).csv → 2025-05-29_18-44-09_DishpatchSubscriptionData_NIklas_Sanitised - subscriptions (2).csv
📅 Reference date (TODAY) for analysis: 30-05-2025


In [2]:
# HELPER FUNCTIONS
##################

def clean_membership_data(df):
    """Clean and prepare membership data for analysis"""
    # Remove very short subscriptions (likely test accounts)
    df['duration_days'] = (pd.to_datetime(df['ended_at_utc']) - pd.to_datetime(df['created_utc'])).dt.days
    df_clean = df[~((df['duration_days'] < 1) & ~(df['status'].isin(['active', 'trialing'])))]
    
    # Remove duplicate signups (within 12 hours)
    df_clean = df_clean.sort_values(['customer_name', 'created_utc'], ascending=[True, False])
    df_clean['time_diff'] = df_clean.groupby('customer_name')['created_utc'].diff()
    df_clean = df_clean[~((df_clean['time_diff'] < pd.Timedelta(hours=12)) & (df_clean['time_diff'].notna()))]
    df_clean = df_clean.sort_values('created_utc', ascending=True)
    
    return df_clean.drop(['duration_days', 'time_diff'], axis=1)



def calculate_real_duration(row):
    """Calculate actual subscription duration"""
    #start_date = row['current_period_start_utc']
    start_date = row['created_utc']
    
    if pd.notna(row['ended_at_utc']):
        end_date = row['ended_at_utc']
    else:
        end_date = row['current_period_end_utc']
    
    return (end_date - start_date).days



def calculate_trial_duration(row):
    """Calculate trial duration"""
    if pd.notna(row['trial_start_utc']) and pd.notna(row['trial_end_utc']):
        start_date = row['trial_start_utc']
        end_date = row['trial_end_utc']
        return (end_date - start_date).days
    return 0

def categorize_duration(duration):
    if duration <= 9:
        return "Early Cancellation (0-9 days)"
    elif 10 <= duration <= 24:
        return "10-day Trial + Refund (10-24 days)"
    elif 25 <= duration <= 34:
        return "20-day Trial + Refund (25-34 days)" 
    elif 35 <= duration <= 55:  # ✅ Élargi
        return "30-day Trial + Refund (35-55 days)"
    elif 360 <= duration <= 420:  # ✅ Élargi
        return "Annual Subscription (360-420 days)"
    elif 730 <= duration <= 770:
        return "Two Year Subscription (730-770 days)"
    else:
        return f"Other ({duration} days)"

In [3]:
# DATA PROCESSING
##################




# Clean abnormal data
analysis_df = clean_membership_data(df)

# Get team test customers with more than 8 occurrences and filter them out
# team_test_customers = analysis_df['customer_name'].value_counts()
# analysis_df = analysis_df[~analysis_df['customer_name'].isin(frequent_customers[frequent_customers >= 8].index)]


# Duration calculations
analysis_df['real_duration'] = analysis_df.apply(calculate_real_duration, axis=1)
# Appliquer la catégorisation
analysis_df['duration_category'] = analysis_df['real_duration'].apply(categorize_duration)
analysis_df['period_duration'] = (analysis_df['current_period_end_utc'] - analysis_df['current_period_start_utc']).dt.days

analysis_df['trial_duration'] = analysis_df.apply(calculate_trial_duration, axis=1)
analysis_df['only_trial'] = analysis_df['period_duration'] == analysis_df['trial_duration']

# Unknown period calculation
analysis_df['unknown_period'] = (analysis_df['current_period_start_utc'] - analysis_df['created_utc']).dt.days

In [17]:
analysis_df['customer_name'].value_counts()
#df['customer_name'].value_counts().head(30)
df_raw['Customer Name'].value_counts().head(30)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20443 entries, 0 to 20442
Data columns (total 34 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  20443 non-null  object 
 1   Customer ID                         20443 non-null  object 
 2   Customer Description                5 non-null      object 
 3   Customer Name                       20443 non-null  object 
 4   Plan                                20443 non-null  object 
 5   Product                             20443 non-null  object 
 6   Product ID                          20443 non-null  object 
 7   Quantity                            20443 non-null  int64  
 8   Currency                            20443 non-null  object 
 9   Cancellation Reason                 13953 non-null  object 
 10  Interval                            20443 non-null  object 
 11  Amount                              20443

In [23]:
len(df_raw['Customer Name'].value_counts()), len(df_raw['Customer ID'].value_counts())

(17738, 19976)

In [24]:
#df_raw['Customer ID'].value_counts()
df_raw['Status'].value_counts()

canceled              13630
active                 6584
trialing                156
past_due                 64
incomplete_expired        9
Name: Status, dtype: int64

In [28]:
# À vérifier immédiatement
print(df_raw[df_raw['Status'] == 'canceled']['Cancellation Reason'].notna().sum(), df_raw[df_raw['Status'].isin(['active', 'trialing'])]['Ended At (UTC)'].isna().sum())

13630 6740


In [29]:
# Vérifier les autres statuts avec raison d'annulation
df_raw[df_raw['Cancellation Reason'].notna() & (df_raw['Status'] != 'canceled')]['Status'].value_counts()

active      308
trialing     15
Name: Status, dtype: int64

In [18]:
df_raw['Cancel At Period End'].value_counts()

False    16878
True      3565
Name: Cancel At Period End, dtype: int64

In [5]:
cancel_at_some_point =df_raw['Canceled At (UTC)'].notna()
ended_at_some_point = df_raw['Ended At (UTC)'].notna()
#len(cancel_at_some_point)
#len(ended_at_some_point)
#len(cancel_at_some_point) + len(ended_at_some_point)
#len(df_raw)


In [6]:
analysis_df['status'].value_counts()

canceled    10608
active       6366
trialing      153
Name: status, dtype: int64

In [7]:
analysis_df = analysis_df[analysis_df['is_gifted_member'] == False]

analysis_df['real_duration'].value_counts().head(30)
#analysis_df[analysis_df['real_duration'] == 0]



10     5577
375    3556
741    1000
365     750
376     566
740     372
6       291
4       256
7       245
5       227
8       220
3       203
9       172
1       162
2       158
25      132
11      130
397      96
31       93
396      75
12       74
761      71
366      61
14       58
13       55
24       47
22       43
21       42
45       40
15       39
Name: real_duration, dtype: int64

In [8]:

# Voir la distribution des catégories
category_counts = analysis_df['duration_category'].value_counts()
category_counts.head(40)

10-day Trial + Refund (10-24 days)      6256
Annual Subscription (360-420 days)      5503
Early Cancellation (0-9 days)           1934
Two Year Subscription (730-770 days)    1732
20-day Trial + Refund (25-34 days)       463
30-day Trial + Refund (35-55 days)       220
Other (276 days)                           3
Other (423 days)                           3
Other (601 days)                           3
Other (771 days)                           3
Other (615 days)                           3
Other (99 days)                            2
Other (108 days)                           2
Other (67 days)                            2
Other (334 days)                           2
Other (268 days)                           2
Other (424 days)                           2
Other (609 days)                           2
Other (77 days)                            2
Other (422 days)                           2
Other (197 days)                           1
Other (256 days)                           1
Other (356

In [9]:
analysis_df[analysis_df['customer_name'] == 'Customer757']
len(analysis_df)

16234

In [10]:
analysis_df[analysis_df['customer_name'] == 'Customer6165']

,customer_name,status,Cancellation Reason,created_utc,start_utc,start_date_utc,current_period_start_utc,current_period_end_utc,trial_start_utc,trial_end_utc,canceled_at_utc,ended_at_utc,is_gifted_member,real_duration,duration_category,period_duration,trial_duration,only_trial,unknown_period
6703,Customer6165,active,NaN,2024-11-04 07:51:00+00:00,2024-11-04 07:51:00+00:00,2024-11-04 07:51:00+00:00,2024-11-04 07:51:00+00:00,2025-11-04 07:51:00+00:00,NaT,NaT,NaT,NaT,False,365,Annual Subscription (360-420 days),365,0,False,0


In [11]:
# analysis_df['is_comeback'] = analysis_df['unknown_period'] > 7



# analysis_df['gap_category'] = pd.cut(
#     analysis_df['unknown_period'], 
#     bins=[0, 1, 7, 30, 90, 365, float('inf')],
#     labels=['immediate', 'next_day', 'week', 'month', 'year', 'more_than_one_year']
# )

# # Analyze comeback patterns
# comeback_stats = analysis_df[analysis_df['is_comeback']].groupby('gap_category').agg({
#     'customer_name': 'count',
#     'real_duration': 'mean',
#     'only_trial': 'sum'
# })

# # mean return delay
# avg_return_delay = analysis_df[analysis_df['is_comeback']]['unknown_period'].mean()

# print(f"avg_return_delay = {avg_return_delay}, comeback_stats = {comeback_stats}")


# # Vérifier les dates suspectes
# suspicious_dates = analysis_df[analysis_df['unknown_period'] > 365]
# print(suspicious_dates[['created_utc', 'current_period_start_utc', 'unknown_period']].head())

# # Vérifier la distribution des dates de création
# print(analysis_df['created_utc'].describe())


# # Identifier les clients probablement migrés
# migrated_customers = analysis_df[
#     (analysis_df['created_utc'].dt.date == pd.to_datetime('2023-09-25').date()) &
#     (analysis_df['unknown_period'] > 300)
# ]

# # Les exclure de l'analyse "comeback" pure
# pure_comebacks = analysis_df[
#     (analysis_df['is_comeback'] == True) &
#     (~analysis_df.index.isin(migrated_customers.index))
# ]

# print(f"migrated_customers = {len(migrated_customers)}, pure_comebacks = {len(pure_comebacks)}") 

# Vérifications supplémentaires nécessaires
# print("Distribution des créations pour les gaps 'year+':")
# year_plus = analysis_df[analysis_df['unknown_period'] > 365]
# print(year_plus['created_utc'].dt.date.value_counts().head(10))

# print("\nRange des unknown_period pour ces clients:")
# print(year_plus['unknown_period'].describe())

In [12]:
# 

In [13]:
analysis_df['customer_name'].value_counts()

Customer17738    1
Customer5809     1
Customer5824     1
Customer5823     1
Customer5821     1
                ..
Customer11757    1
Customer11756    1
Customer11755    1
Customer11754    1
Customer1        1
Name: customer_name, Length: 16234, dtype: int64

In [14]:
df['created_utc'].min()

Timestamp('2023-09-25 09:04:00+0000', tz='UTC')

In [15]:
df['customer_name'].value_counts()


Customer5019     33
Customer8877     17
Customer2832     11
Customer13370    11
Customer2742      9
                 ..
Customer16739     1
Customer10765     1
Customer10764     1
Customer10763     1
Customer1         1
Name: customer_name, Length: 17738, dtype: int64

In [16]:


# Define conversion and cancellation logic
analysis_df['paid_after_trial'] = (
    (analysis_df['status'] == 'active') |
    ((analysis_df['status'] == 'canceled') &
     (analysis_df['only_trial'] == False) &
     (analysis_df['canceled_at_utc'].isna())) |
    ((analysis_df['status'] == 'canceled') &
     (analysis_df['canceled_at_utc'] > analysis_df['trial_end_utc']))
)

analysis_df['cancel_during_trial'] = (
    ((analysis_df['status'] == 'canceled') &
     (analysis_df['canceled_at_utc'] <= analysis_df['trial_end_utc'])) |
    ((analysis_df['status'] == 'trialing') & 
     (analysis_df['canceled_at_utc'].notna()))
)

# Cas 1: Clients avec essai - remboursés dans les 14 jours après l'essai
refund_after_trial = (
    (analysis_df['status'] == 'canceled') &
    (analysis_df['trial_end_utc'].notna()) &  # A eu un essai
    (analysis_df['only_trial'] == False) &
    (analysis_df['cancel_during_trial'] == False) &
    (analysis_df['canceled_at_utc'] < analysis_df['trial_end_utc'] + pd.Timedelta(days=14)) &
    (analysis_df['canceled_at_utc'] > analysis_df['trial_end_utc'])  # Annulé APRÈS l'essai
)

# Cas 2: Clients sans essai - remboursés dans les 14 jours après le début de facturation
refund_no_trial = (
    (analysis_df['status'] == 'canceled') &
    (analysis_df['trial_end_utc'].isna()) &  # Pas d'essai
    (analysis_df['canceled_at_utc'] < analysis_df['current_period_start_utc'] + pd.Timedelta(days=14))
)

# Combinaison des deux cas
analysis_df['was_refund'] = refund_after_trial | refund_no_trial

analysis_df['in_churn_period'] = (
    ((analysis_df['status'] == 'active') &
     (analysis_df['trial_end_utc'] + pd.Timedelta(days=14) >= reference_date)) |
    ((analysis_df['status'] == 'active') &
     (analysis_df['current_period_start_utc'] + pd.Timedelta(days=14) >= reference_date))
)

analysis_df['end_soon'] = (
    (analysis_df['status'] == 'active') &
    (analysis_df['current_period_end_utc'] + pd.Timedelta(days=14) >= reference_date) &
    (analysis_df['current_period_end_utc'] > reference_date)

SyntaxError: incomplete input (2800851125.py, line 48)